In [1]:
import sys

sys.path.append("..")
from piece import piece

piece.start(should_send_to_score=False)

In [83]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.spectrum import ring_modulate

import random
SOUND_PATH = "/Users/danielstahl/Documents/Music/Pieces/Concrete Music/Concrete Music 11/sounds/Concrete Music 11_sounds"

GLASS_TONE_SOUND = "glass_tone"
# 845, 951, 1060, 1162, 1327, 1466
GLASS_HIT_SOUND = "glass_hit"
# 216, 654, 846.5, 982, 1053.5
VASE_TONE_SOUND = "vase_tone"
# 313, 

piece.reset()
(piece.synth_player
 .add_sound(GLASS_TONE_SOUND, f"{SOUND_PATH}/Glass Tone.flac", 0.119, 4.097)
 .add_sound(GLASS_HIT_SOUND, f"{SOUND_PATH}/Class Hit.flac", 0.086, 0.186)
 .add_sound(VASE_TONE_SOUND, f"{SOUND_PATH}/Vase Tone.flac", 0.043, 0.463)
 .start())

class GlassToneRingHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        ring_tones = [845, 951, 1060, 1162, 1327, 1466]
        ring_freq = ring_tones[patch_arguments.note]
        #ring_freq = random.choice([216, 654, 846.5, 982, 1053.5])                              
        match patch_arguments.octave:
            case 2:
                (piece.synth_player.note()
                    .sound_mono(GLASS_TONE_SOUND, 1, piece.synth_player.control_instruments.static_control(1))
                    .ring_modulate(piece.synth_player.control_instruments.static_control(ring_freq))
                    .pan(piece.synth_player.control_instruments.static_control(0))
                    .play(patch_arguments.start))
            case 3:
                duration = 5                                
                (piece.synth_player.note()
                    .sound_mono(GLASS_TONE_SOUND, 1, piece.synth_player.control_instruments.static_control(1))                    
                    .ring_modulate(piece.synth_player.control_instruments.static_control(ring_freq))
                    .pan(piece.synth_player.control_instruments.static_control(0))
                    .stereo_free_reverb(piece.synth_player.control_instruments.sine_control(0, 5), mix=0.33, room=0.5, damp=0.8) \
                    .play(patch_arguments.start, duration))
                
                
class NoiseNoteHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        match patch_arguments.note:
            case 0:
                ring_freq = random.choice([845, 951, 1060, 1162, 1327, 1466])  
                #ring_freq = random.choice([216, 654, 846.5, 982, 1053.5])                              
                (piece.synth_player.note()
                    .sound_mono(GLASS_TONE_SOUND, 1, piece.synth_player.control_instruments.static_control(1))
                    .ring_modulate(piece.synth_player.control_instruments.static_control(ring_freq))
                    .pan(piece.synth_player.control_instruments.static_control(0))
                    .play(patch_arguments.start))
            case 1:    
                ring_freq = random.choice([845, 951, 1060, 1162, 1327, 1466])              
                #ring_freq = random.choice([216, 654, 846.5, 982, 1053.5])                
                (piece.synth_player.note()
                    .sound_mono(GLASS_HIT_SOUND, 1, piece.synth_player.control_instruments.static_control(1))                
                    .ring_modulate(piece.synth_player.control_instruments.static_control(ring_freq))
                    .pan(piece.synth_player.control_instruments.static_control(0))
                    .play(patch_arguments.start))
            case 2:
                (piece.synth_player.note()
                    .sound_mono(VASE_TONE_SOUND, 1, piece.synth_player.control_instruments.static_control(1))
                    .pan(piece.synth_player.control_instruments.static_control(0))
                    .play(patch_arguments.start))
            case 3:
                duration = 5                
                ring_freq = random.choice([845, 951, 1060, 1162, 1327, 1466])                
                (piece.synth_player.note()
                    .sound_mono(GLASS_TONE_SOUND, 1, piece.synth_player.control_instruments.static_control(1))                    
                    .ring_modulate(piece.synth_player.control_instruments.static_control(ring_freq))
                    .pan(piece.synth_player.control_instruments.static_control(0))
                    .stereo_free_reverb(piece.synth_player.control_instruments.sine_control(0, 5), mix=0.33, room=0.5, damp=0.8) \
                    .play(patch_arguments.start, duration))
            case 4:
                duration = 5          
                ring_freq = random.choice([216, 654, 846.5, 982, 1053.5])                    
                (piece.synth_player.note()
                    .sound_mono(GLASS_HIT_SOUND, 1, piece.synth_player.control_instruments.static_control(1))     
                    .ring_modulate(piece.synth_player.control_instruments.static_control(ring_freq))               
                    .pan(piece.synth_player.control_instruments.static_control(0))
                    .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, 2), roomsize=50, revtime=duration) \
                    .play(patch_arguments.start, duration))
            case 5:
                duration = 5
                (piece.synth_player.note()
                    .sound_mono(VASE_TONE_SOUND, 0.2, piece.synth_player.control_instruments.static_control(1))
                    .pan(piece.synth_player.control_instruments.static_control(0))
                    .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, 1), roomsize=50, revtime=duration) \
                    .play(patch_arguments.start, duration))

my_handler = GlassToneRingHandler(piece.supercollider_client)
piece.receiver.set_note_handler(my_handler)                    



In [ ]:
piece.stop()

In [68]:
ring_modulate(1060, 845)

215 / 845 



0.25443786982248523